<a href="https://colab.research.google.com/gist/youlive789/7b704c29d6d144bdc837eda85aff7682/textsummariation-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라이브러리 설치

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

# !git clone https://github.com/huggingface/nlp.git
!pip install git+git://github.com/huggingface/nlp.git@0.4.0
!pip install --force-reinstall pyarrow==0.16.0

!pip install rouge_score rouge_score

## 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## 데이터 준비
준비한 데이터를 판다스로 불러오고 huggingface의 nlp 라이브러리를 이용하여 학습 데이터를 만들어줍니다.
  
  
**주의점**
 - 이 노트에서 사용한 데이터는 매우 작습니다. 실제 테스트 하실 때에는 충분히 데이터를 확보해주세요.

In [ ]:
import pandas as pd

# input raw data
raw_data_train = pd.read_csv("/content/gdrive/MyDrive/Colab_Notebooks/capstone-29/news-topic/summarization_data/json_to_csv_train_신문기사.csv").drop(['Unnamed: 0'],axis=1)
raw_data_valid = pd.read_csv("/content/gdrive/MyDrive/Colab_Notebooks/capstone-29/news-topic/summarization_data/json_to_csv_valid_신문기사.csv").drop(['Unnamed: 0'],axis=1)
raw_data_valid.head()

,media,id,article_orginal,abstractive,extractive
0,한국경제,340626877,"['[ 박재원 기자 ] \'대한민국 5G 홍보대사\'를 자처한 문재인 대통령은 ""넓...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"
1,한국경제,340626896,"['] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다.', '바른미래당이 ...",8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...,"[2, 1, 6]"
2,한국경제,340626904,"['[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀.', ""지난 3일 한국이 세계...",지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...,"[1, 5, 8]"
3,한국경제,340627450,['] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...,"[0, 1, 2]"
4,한국경제,340627465,"['[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조...",주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...,"[1, 3, 4]"


In [ ]:
import pandas as pd

df_train = [
]
col = ['short', 'long']
df_train = pd.DataFrame(df_train, columns=col)
df_train

,short,long


In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(raw_data_train))):
  data = raw_data_train.loc[i]
  short_data = data[3]
  long_data = data[2][2:-2]

  table = long_data.maketrans({ ',': '', "'": ''})
  long_data = long_data.translate(table)

  new_data = {'short' : short_data, 'long' : long_data}
  df_train = df_train.append(new_data, ignore_index=True)

100%|██████████| 243983/243983 [48:27<00:00, 83.92it/s]


In [ ]:
df_train.to_csv('/content/gdrive/MyDrive/Colab_Notebooks/capstone-29/news-topic/summarization_data/csv_train.csv', encoding='utf-8-sig')

In [ ]:
import pandas as pd

df_valid = [
]
col = ['short', 'long']
df_valid = pd.DataFrame(df_valid, columns=col)
df_valid

,short,long


In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(raw_data_valid))):
  data = raw_data_valid.loc[i]
  short_data = data[3]
  long_data = data[2][2:-2]

  table = long_data.maketrans({ ',': '', "'": ''})
  long_data = long_data.translate(table)

  new_data = {'short' : short_data, 'long' : long_data}
  df_valid = df_valid.append(new_data, ignore_index=True)

100%|██████████| 30122/30122 [02:56<00:00, 170.99it/s]


In [ ]:
df_valid.to_csv('/content/gdrive/MyDrive/Colab_Notebooks/capstone-29/news-topic/summarization_data/csv_valid.csv', encoding='utf-8-sig')

## 데이터 세팅

In [ ]:
import pandas as pd

df_train = pd.read_csv("/content/gdrive/MyDrive/csv_train.csv").drop(['Unnamed: 0'],axis=1)
df_valid = pd.read_csv("/content/gdrive/MyDrive/csv_valid.csv").drop(['Unnamed: 0'],axis=1)

In [ ]:
print(len(df_train))
print(len(df_valid))

243983
60244


In [ ]:
df_train_dn = df_train.dropna(how='any')
df_valid_dn = df_valid.dropna(how='any')

print(len(df_train_dn))
print(len(df_valid_dn))

234702
60244


In [ ]:
pip install nlp

     |████████████████████████████████| 1.7 MB 4.3 MB/s 
     |████████████████████████████████| 212 kB 20.9 MB/s 


In [ ]:
import nlp
train_dataset = nlp.Dataset.from_pandas(df_train_dn)
val_dataset = nlp.Dataset.from_pandas(df_valid_dn)

## 학습데이터 전처리 및 모델정의

In [ ]:
 import torch
 from transformers import EncoderDecoderModel, BertTokenizer, Trainer, TrainingArguments

Bert multilingual 모델을 encoder, decoder로 연결해줍니다. 또한 bert multilingual용 토크나이저도 생성해줍니다.

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-uncased', 'bert-base-multilingual-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

map_to_encoder_decoder_inputs : 문장을 bert에 입력할 수 있도록 전처리를 수행합니다.  
compute_metrics : rouge2 점수를 통해 성능을 측정합니다.

tokenizer 안의 max_length를 통해 input과 output의 길이를 조정할 수 있습니다.

In [ ]:
from sklearn.metrics import accuracy_score

def map_to_encoder_decoder_inputs(batch):
    inputs = tokenizer(batch["long"], padding="max_length", truncation=True, max_length=512) # 요약하고자 하는 문장
    outputs = tokenizer(batch["short"], padding="max_length", truncation=True, max_length=512) # 요약이 된 정답 문장

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()

    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]
    batch["decoder_attention_mask"] = outputs.attention_mask

    assert all([len(x) == 512 for x in inputs.input_ids])
    assert all([len(x) == 512 for x in outputs.input_ids])

    return batch

def compute_metrics(pred):
  labels_ids = pred.label_ids
  pred_ids = pred.predictions
  preds = pred.predictions.argmax(-1)

  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

  acc = accuracy_score(labels_ids, preds)
  # print(round(rouge_output.recall, 4))

  return {
      "accuray": acc,
      "rouge2_precision": round(rouge_output.precision, 4),
      "rouge2_recall": round(rouge_output.recall, 4),
      "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
  }

학습데이터와 검증데이터를 나누어 전처리를 수행합니다.

In [ ]:
batch_size_train = 32

train_dataset = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size_train, remove_columns=["short", "long"],
)
train_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

  0%|          | 0/14669 [00:00<?, ?it/s]

In [ ]:
batch_size_valid = 64

val_dataset = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size_valid, remove_columns=["short", "long"],
)
val_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

  0%|          | 0/3766 [00:00<?, ?it/s]

In [ ]:
# from transformers import AdamW

# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]

# optimizer = AdamW(
#     optimizer_grouped_parameters,
#     lr=0.001,
#     eps=1e-06,
#     correct_bias=True,
#     no_deprecation_warning=True
#     )

In [ ]:
# Trainer - loss function 추가

class MyTrainer(Trainer):
      # loss_name 이라는 인자를 추가로 받아 self에 각인 시켜줍니다.
      def __init__(self, loss_name, *args, **kwargs):
          super().__init__(*args, **kwargs)
          self.loss_name= loss_name # 각인!

      def compute_loss(self, model, inputs, return_outputs=False):
        # config에 저장된 loss_name에 따라 다른 loss 계산 
        if self.loss_name == 'CrossEntropy':
            # lossname이 CrossEntropy 이면, custom_loss에 torch.nn.CrossEntropyLoss()를 선언(?) 해줍니다.
            custom_loss = torch.nn.CrossEntropyLoss()
                      
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None

        outputs = model(**inputs)

        if labels is not None:
            #loss를 계산 하던 부분에 custom_loss를 이용해 계산하는 코드를 넣어 줍니다!
            #원본 코드를 보시면 output[0]가 logit 임을 알 수 있습니다!
            loss = custom_loss(outputs[0], labels)
        else:
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
        return (loss, outputs) if return_outputs else loss

모델에 필요한 환경설정을 해줍니다.

In [ ]:
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
rouge = nlp.load_metric("rouge")
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.max_length = 256
model.config.min_length = 512
model.config.no_repeat_ngram_size = 3
model.early_stopping = True
model.length_penalty = 2.0
model.num_beams = 4

Downloading:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

In [ ]:
print(device)

## 모델학습

trainer를 통해 모델을 학습시킵니다.

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/gdrive/MyDrive/results_2",
    num_train_epochs=3,
    per_device_train_batch_size=batch_size_train,
    per_device_eval_batch_size=batch_size_valid,
    warmup_steps=500,
    weight_decay=0.01,
    do_train=True,
    do_eval=True,
    logging_steps=30,
    save_steps=5000,
    eval_steps=500,
    overwrite_output_dir=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# trainer = MyTrainer(
#       loss_name='CrossEntropy',
#       model=model,
#       args=training_args,
#       train_dataset=train_dataset,
#       eval_dataset=val_dataset,
#       compute_metrics=compute_metrics,
#       # optimizers=(optimizer, transformers.get_linear_schedule_with_warmup())
#       )

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 234702
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 44007
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:532: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based o

Step,Training Loss
30,2.794100
60,0.829400
90,0.151700
120,0.032800
150,0.017300
180,0.010700
210,0.006900
240,0.003100
270,0.002900
300,0.002800


Saving model checkpoint to /content/gdrive/MyDrive/model/checkpoint-500
Configuration saved in /content/gdrive/MyDrive/model/checkpoint-500/config.json
Model weights saved in /content/gdrive/MyDrive/model/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:532: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to /content/gdrive/MyDrive/model/checkpoint-1000
Configuration saved in /content/gdrive/MyDrive/model/checkpoint-1000/config.json
Model weights saved in /content/gdrive/MyDrive/model/checkpoint-1

KeyboardInterrupt: ignored

In [ ]:
trainer.evaluate()
# saving the fine tuned model & tokenizer
model_path = "/content/gdrive/MyDrive/models"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in /content/gdrive/MyDrive/models/config.json
Model weights saved in /content/gdrive/MyDrive/models/pytorch_model.bin
tokenizer config file saved in /content/gdrive/MyDrive/models/tokenizer_config.json
Special tokens file saved in /content/gdrive/MyDrive/models/special_tokens_map.json


('/content/gdrive/MyDrive/models/tokenizer_config.json',
 '/content/gdrive/MyDrive/models/special_tokens_map.json',
 '/content/gdrive/MyDrive/models/vocab.txt',
 '/content/gdrive/MyDrive/models/added_tokens.json')

In [ ]:
torch.save(model, '/content/gdrive/MyDrive/batch_32_64_model_epoch_3.pt')

성능을 테스트 해봅시다.

In [ ]:
sentence = """
그런데 서로 다른 키 값에 대해 같은 방 번호가 나온다면 어떻게 될까? 키를 통해 얻으려고 하는 값을 얻지 못하거나, 
엉뚱한 키가 삭제되거나 하는 문제가 발생하게 된다. 이런 경우를 "해시 충돌"이라 하며, 해시 충돌을 최대한으로 줄이는 해시 함수의 설계가 해시 테이블의 핵심이라고 볼 수 있다. 
충돌이 일어나도 해결할 수 있는 방법은 두 가지가 있다. 가장 많이 쓰이는 방법은 체이닝이다. 해시 함수를 통해 얻어지는 방 번호에 해당하는 연결 리스트 등의 자료구조 만들어 놓고, 
충돌이 일어나면 키 값을 통해 연결리스트를 순회하는 방법이다. 두 번째 방법은 개방 번지화이다. 
이 방법은 비어있는 공간을 찾아서 할당하는 방법이다. 이번 포스팅에서는 이 방법으로 해시맵을 구현했다.
"""

if torch.cuda.is_available():
  d = "cuda"
else:
  d = "cpu"

input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0).to(torch.device(d))
generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.pad_token_id)
tokenizer.decode(generated[0])

Setting `pad_token_id` to `eos_token_id`:102 for open-end generation.


'[PAD] [CLS] [CLS] [CLS] kb kb kb mb mb mb kb kb16161615151516161717171616141414161613141415151414131413131415141516141317141417141315141316161214141214'

In [ ]:
# inference

from transformers import BertForSequenceClassification, BertTokenizerFast
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups

model_path = "20newsgroups-bert-base-uncased"
max_length = 512

def read_20newsgroups(test_size=0.2):
  dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers", "footers", "quotes"))
  documents = dataset.data
  labels = dataset.target
  return train_test_split(documents, labels, test_size=test_size), dataset.target_names


(train_texts, valid_texts, train_labels, valid_labels), target_names = read_20newsgroups()

model = BertForSequenceClassification.from_pretrained(model_path, num_labels=len(target_names)).to("cuda")
tokenizer = BertTokenizerFast.from_pretrained(model_path)

def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return target_names[probs.argmax()]

# Example #1
text = """With the pace of smartphone evolution moving so fast, there's always something waiting in the wings. 
No sooner have you spied the latest handset, that there's anticipation for the next big thing. 
Here we look at those phones that haven't yet launched, the upcoming phones for 2021. 
We'll be updating this list on a regular basis, with those device rumours we think are credible and exciting."""
print(get_prediction(text))
# Example #2
text = """
A black hole is a place in space where gravity pulls so much that even light can not get out. 
The gravity is so strong because matter has been squeezed into a tiny space. This can happen when a star is dying.
Because no light can get out, people can't see black holes. 
They are invisible. Space telescopes with special tools can help find black holes. 
The special tools can see how stars that are very close to black holes act differently than other stars.
"""
print(get_prediction(text))